### Get all country names in list

In [ ]:
import geonamescache
import re
gc = geonamescache.GeonamesCache()

countries = list(gc.get_countries_by_names().keys())

print(countries)

### Get all city names in list

#### Make equivalent list with word boundary regex

In [ ]:
import geonamescache
import re
gc = geonamescache.GeonamesCache()

cities = gc.get_cities()
city_names = [cities[key]['name'] for key in list(cities.keys())]

city_name_res = [re.compile('\\b{}\\b'.format(name), flags=re.IGNORECASE) for name in city_names]
city_name_res

### Get all US state names in list
#### Make equivalent list with word boundary regex

In [ ]:
import geonamescache
import re
gc = geonamescache.GeonamesCache()

us_states = gc.get_us_states()

us_state_names = [us_states[key]['name'] for key in list(us_states.keys())]
us_state_names

us_state_name_res = [re.compile('\\b{}\\b'.format(name), flags=re.IGNORECASE) for name in us_state_names]
us_state_name_res

### Get all US counties as dicts (with name and state keys) in list
#### Make equivalent list with word boundary regex (with and without " County")

In [ ]:
import geonamescache
import re
gc = geonamescache.GeonamesCache()

us_counties = gc.get_us_counties()
# Keep as is to have name and state
us_counties

us_county_res = [re.compile('\\b({}|{})\\b'.format(county_dict['name'], county_dict['name'][:-7]), flags=re.IGNORECASE) for county_dict in us_counties]
us_county_res

### Get headlines in list

In [ ]:
import os

headlines_path = os.path.join('..', 'data', 'headlines.txt')
headlines_file = open(headlines_path, "r")
headlines_str = headlines_file.read()
headlines_list = headlines_str.split('\n')
headlines_list